In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
url='https://chipublib.bibliocommons.com/events/rss/all'
program_list={}
my_dict = {"program_name":[],
           "program_description":[],
           "program_url":[],
           "virtual":[],
           "registration_url":[],
           "start_date":[],
           "end_date":[],
           "program":[],
           "category":[],
          "audience":[],
          "language":[],
          "location_name":[],
          "location_address":[]}

In [3]:
r=requests.get(url)
soup=bs(r.text, features='xml')
#         return print('CPL scrape successful: ',r.status_code)
programs=soup.find_all('item')

In [4]:
for p in programs:
            title=p.find('title').text
            description=p.find('description').text.strip()
            link=p.find('link').text
            virtual=p.find('bc:is_virtual').text
            start_date=p.find('bc:start_date_local').text
            end_date=p.find('bc:end_date_local').text
            program=getattr(p.find('category',domain='Program'), 'text', None)
            category=getattr(p.find('category',domain='Type'), 'text', None)
            audience=p.find_all('category',domain='Audience')
            language=p.find_all('category',domain='Language')
#             location_name=p.find_all('bc:location','bc:name')
            location_number=getattr(p.find('bc:number'), 'string', None)
            location_street=getattr(p.find('bc:street'), 'string', None)
            location_address=f'{location_number} {location_street}'
            location_city=p.find('bc:city')
            location_zip=p.find('bc:zip')
            location_state=p.find('bc:state')
            
            my_dict["program_name"].append(title)
            my_dict["program_description"].append(description)
            my_dict["program_url"].append(link)
            my_dict["registration_url"].append(link)
            my_dict["virtual"].append(virtual)
            my_dict["start_date"].append(start_date)
            my_dict["end_date"].append(start_date)
            my_dict["program"].append(program)
            my_dict["category"].append(category)
            my_dict["audience"].append(audience)
            my_dict["language"].append(language)
#             my_dict["location_name"].append(location_name)
            my_dict["location_address"].append(location_address)
            
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)

In [5]:
type(my_dict)

dict

In [6]:
 my_dict["location_name"]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [7]:
type(program)

NoneType

In [8]:
df = pd.DataFrame(my_dict)

In [9]:
df.head(30)

,program_name,program_description,program_url,virtual,registration_url,start_date,end_date,program,category,audience,language,location_name,location_address
0,Take and Make Kit: Decorative Clay Dish,"<figure><a href=""https://www.flickr.com/photos...",https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-06T09:30,2021-10-06T09:30,Polish American Heritage,"Crafts, Games and Play","[[Adults], [Seniors]]",[[English]],[],400 S. State Street
1,Cantos y Cuentos Bilingual Story Time,<p>Join the Thomas Hughes Children's Library s...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:00,2021-10-06T10:00,None,Story Time,"[[Babies], [Preschoolers], [Toddlers]]","[[English], [Español]]",[],None None
2,Story Time,<p>Join Sulzer Regional Library's&nbsp;story t...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:00,2021-10-06T10:00,None,Story Time,"[[Babies], [Preschoolers], [Toddlers]]",[[English]],[],None None
3,Stories and Songs,<p>Join us for Stories and Songs&nbsp;in this ...,https://chipublib.bibliocommons.com/events/613...,false,https://chipublib.bibliocommons.com/events/613...,2021-10-06T10:30,2021-10-06T10:30,None,Story Time,"[[Kids], [Toddlers]]",[[English]],[],6348 S. Archer Avenue
4,Mother Goose on the Loose,<p>Mother Goose on the Loose is a story time t...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:30,2021-10-06T10:30,None,Story Time,"[[Babies], [Toddlers]]",[[English]],[],None None
5,Family Story Time in The Not-So-Secret Garden,<p>Join Ms. Evelyn and Ms. Sara from the Uptow...,https://chipublib.bibliocommons.com/events/612...,false,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:30,2021-10-06T10:30,None,Story Time,"[[Kids], [Preschoolers]]",[[English]],[],929 W. Buena Avenue
6,Preschool Story Time,<p>Join our Jefferson Park Branch story time e...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-06T10:30,2021-10-06T10:30,None,Story Time,[[Preschoolers]],[[English]],[],None None
7,Preschool Story Time,<p>Have fun and build literacy skills at the s...,https://chipublib.bibliocommons.com/events/613...,false,https://chipublib.bibliocommons.com/events/613...,2021-10-06T10:30,2021-10-06T10:30,None,Story Time,[[Preschoolers]],[[English]],[],6083 N. Northwest Highway
8,Preschool Story Time,<p>Join us&nbsp;for Preschool Story Time&nbsp;...,https://chipublib.bibliocommons.com/events/614...,true,https://chipublib.bibliocommons.com/events/614...,2021-10-06T10:30,2021-10-06T10:30,None,Story Time,"[[Preschoolers], [Toddlers]]",[[English]],[],None None
9,Tiny Tykes Story Time,<p>Join Mrs. Susie from Chicago Lawn Branch fo...,https://chipublib.bibliocommons.com/events/608...,false,https://chipublib.bibliocommons.com/events/608...,2021-10-06T11:00,2021-10-06T11:00,None,Story Time,"[[Preschoolers], [Toddlers]]",[[English]],[],6120 S. Kedzie Avenue


In [10]:
# def cpl_programs(url):
#     program_list=[]
#     try:
#         r=requests.get(url)
#         soup=BeautifulSoup(r.content, features='xml')
# #         return print('CPL scrape successful: ',r.status_code)
#         programs = soup.findAll('item')
    
#         for p in programs:
#             title=p.find('title').text
#             link=p.find('link').text
#             start_date=p.find('bc:start_date_local').text
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)
        

#         return print(program_list)
#     except Exception as e:
#         print('FAILED. See exception: ')
#         print(e)
        


In [11]:
# print('Starting scrape')
# cpl_programs(url)
# print('Finished')

In [12]:
# test=cpl_programs(url)